In [2]:
import torch
import librosa
import numpy as np
from transformers import Wav2Vec2ForCTC, Wav2Vec2Tokenizer

# Load pre-trained model and tokenizer
model = Wav2Vec2ForCTC.from_pretrained("facebook/wav2vec2-large-960h-lv60-self")
tokenizer = Wav2Vec2Tokenizer.from_pretrained("facebook/wav2vec2-large-960h-lv60-self")



Some weights of Wav2Vec2ForCTC were not initialized from the model checkpoint at facebook/wav2vec2-large-960h-lv60-self and are newly initialized: ['wav2vec2.masked_spec_embed']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
The tokenizer class you load from this checkpoint is not the same type as the class this function is called from. It may result in unexpected tokenization. 
The tokenizer class you load from this checkpoint is 'Wav2Vec2CTCTokenizer'. 
The class this function is called from is 'Wav2Vec2Tokenizer'.


In [16]:
# Load WAV file and extract features
audio_file = "transcript.wav"
signal, sample_rate = librosa.load(audio_file, sr=16000)
inputs = tokenizer(signal, return_tensors="pt", padding=True)

# Pass features through the model to get predicted transcriptions
with torch.no_grad():
    outputs = model(inputs.input_values, attention_mask=inputs.attention_mask)

predicted_ids = np.argmax(outputs.logits, axis=-1)
transcription = tokenizer.batch_decode(predicted_ids)[0]
print(transcription)

# Identify timestamps of 'm' phoneme with confidence above a certain threshold
m_indices = [i for i, (ph, conf) in enumerate(zip(transcription, outputs.logits.softmax(-1).max(-1).values)) if ph == 'M' and conf > 0.7]
m_timestamps = [(idx * 0.02, (idx+1) * 0.02) for idx in m_indices]  # assuming frame shift of 0.02 seconds

print(m_timestamps)

WE'RE ENTERING AN ERA IN WHICH OUR ENEMIES CAN MAKE IT LOOK LIKE ANYONE IS SAYING ANYTHING AT ANY POINT IN TIME EVEN IF THEY WOULD NEVER SAY THOSE THINGS SO FOR INSTANCE THEY COULD HAVE ME SAY THINGS LIKE I DON'T KNOW KILLMONGER WAS RIGHT OR BEN CARSON IS IN THE SUNKIN PLACE OR HOW ABOUT THIS SIMPLY PRESIDENT TRUMP IS A TOTAL AND COMPLETE DIPSHIT NOW YOU SEE I WOULDT NEVER SAY THESE THINGS AT LEAST NOT IN A PUBLIC ADDRESS BUT SOMEONE ELSE WOULD SOMEONE LIKE GORDAN PEEL THIS IS A DANGEROUS TIME MOVING FORWARD WE NEED TO BE MORE VIGILANT WITH WHAT WE TRUST FROM THE INTERNET AT'S A TIME WHEN WE NEED TO RELY ON TRUSTED NEW SOURCES MAY SOUND BASIC BUT HOW WE MOVE FORWARD IN THE AGE OF INFORMATION ITIS GOING TO BE THE DIFFERENCE BETWEEN WHETHER WE SURVIVE OR WHETHER WE BECOME SOME KIND OF FUCKED UP DISTOPIA THANK YOU STAY WOLK PITCHES
[]


In [15]:
import deepspeech
import numpy as np
import wave

# Load the DeepSpeech model
model = deepspeech.Model('deepspeech-0.9.3-models.pbmm')

# Load the audio file
audio_file = wave.open('transcript.wav', 'rb')
audio_data = audio_file.readframes(audio_file.getnframes())

# Convert the audio data to a NumPy array
audio_data_np = np.frombuffer(audio_data, np.int16)

# Transcribe the audio file
transcript = model.stt(audio_data_np)
print(transcript)

# Extract the phonemes with closed mouth shapes
closed_mouth_phonemes = ['M', 'B', 'P']
phonemes = transcript.split(' ')
closed_mouth_phonemes_detected = [p for p in phonemes if p in closed_mouth_phonemes]

# Print the detected phonemes
print('Closed mouth phonemes detected: ', closed_mouth_phonemes_detected)


In [19]:
import torch
import librosa
import numpy as np
from transformers import Wav2Vec2Tokenizer, Wav2Vec2ForCTC

# Load the audio file and resample it to the required sample rate
audio_file = 'transcript.wav'
sr = 16000
audio, _ = librosa.load(audio_file, sr=sr)

# Split the audio file into segments of 5 seconds
segment_len = 60 * sr
segments = [audio[i:i+segment_len] for i in range(0, len(audio), segment_len)]

# Load the Wav2Vec2 tokenizer and model
tokenizer = Wav2Vec2Tokenizer.from_pretrained("facebook/wav2vec2-base-960h")
model = Wav2Vec2ForCTC.from_pretrained("facebook/wav2vec2-base-960h")

# Define the phonemes of interest
target_phonemes = ['M', 'B', 'P']

# Initialize the output array for the phoneme times
phoneme_times = {ph: [] for ph in target_phonemes}

# Iterate over the audio segments and predict the phonemes
for i, segment in enumerate(segments):
    input_values = tokenizer(segment, return_tensors='pt').input_values
    with torch.no_grad():
        logits = model(input_values).logits
    predicted_ids = torch.argmax(logits, dim=-1)
    predicted_phonemes = tokenizer.batch_decode(predicted_ids)[0]
    print(len(predicted_phonemes))

    # Add the times of occurrence of the target phonemes to the output array
    for ph in target_phonemes:
        ph_indices = [j for j, p in enumerate(predicted_phonemes) if p == ph]
        ph_times = [((i*segment_len + j) * (1.0/sr)) for j in ph_indices]
        end_val = ((i*segment_len + len(predicted_phonemes)) * (1.0/sr))
        end_val = (int(end_val) + ((end_val - int(end_val))*1000))
        print(type(end_val))

        for k in range(len(ph_times)):
            num = ph_times[k]
            ph_times[k] = format(((int(num) + ((num - int(num))*1000)) * (segment_len/sr) * (1.0/end_val)), ".3f")
        phoneme_times[ph].extend(ph_times)

# Print the output phoneme times
print('Phoneme times:')
for ph in target_phonemes:
    print(f'{ph}: {phoneme_times[ph]}')


The tokenizer class you load from this checkpoint is not the same type as the class this function is called from. It may result in unexpected tokenization. 
The tokenizer class you load from this checkpoint is 'Wav2Vec2CTCTokenizer'. 
The class this function is called from is 'Wav2Vec2Tokenizer'.
Some weights of Wav2Vec2ForCTC were not initialized from the model checkpoint at facebook/wav2vec2-base-960h and are newly initialized: ['wav2vec2.masked_spec_embed']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


702
<class 'float'>
<class 'float'>
<class 'float'>
134
<class 'float'>
<class 'float'>
<class 'float'>
Phoneme times:
M: ['3.248', '4.017', '9.402', '15.983', '18.632', '24.701', '26.239', '27.949', '36.325', '38.034', '42.051', '42.308', '44.786', '47.863', '50.171', '54.188', '56.496', '59.231', '56.600', '56.874']
B: ['20.342', '23.675', '34.701', '35.812', '44.530', '55.043', '55.556', '53.144', '54.132', '56.380', '59.616']
P: ['8.462', '22.735', '24.786', '25.128', '26.325', '28.034', '28.718', '34.530', '39.744', '57.916', '58.300']
